# Lab 5.3.1 The Validation Set Approach

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

df = pd.read_csv("C:\\Users\\diteh\\Documents\\Biomedicinsk Teknologi\\3. semester\\Decision Support System\\Lab\\Auto.csv", na_values='?')
df = df.dropna().reset_index(drop = True)
df.head(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198.0,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220.0,4354,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215.0,4312,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225.0,4425,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190.0,3850,8.5,70,1,amc ambassador dpl


In [2]:
# Dividing  into test and training sets. Equal size. 
X_train, X_test, Y_train, Y_test = train_test_split(df.horsepower,df.mpg, test_size = 0.5, random_state= 42)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
mpg             392 non-null float64
cylinders       392 non-null int64
displacement    392 non-null float64
horsepower      392 non-null float64
weight          392 non-null int64
acceleration    392 non-null float64
year            392 non-null int64
origin          392 non-null int64
name            392 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 27.6+ KB


In [3]:
X_train = np.asarray(X_train).reshape(-1,1)
Y_train = np.asarray(Y_train).reshape(-1,1)
X_test = np.asarray(X_test).reshape(-1,1)
Y_test = np.asarray(Y_test).reshape(-1,1)

reg = LinearRegression().fit(X_train,Y_train)

In [4]:
# 1. order
pred = reg.predict(np.asarray(df.horsepower).reshape(-1,1))
mse = mean_squared_error(np.asarray(df.mpg).reshape(-1,1),pred) 
print(mse) 

24.36780369448904


In [5]:
# 2. order = polynominal  
model = Pipeline([('poly', PolynomialFeatures(degree=2)),('linear', LinearRegression(fit_intercept=False))])
model = model.fit(X_train, Y_train)
model.named_steps['linear'].coef_

pred2 = model.predict(np.asarray(df.horsepower).reshape(-1,1))
mse2 = mean_squared_error(np.asarray(df.mpg).reshape(-1,1),pred2)
print(mse2)

19.528868097422244


In [6]:
# 3. order = cubic 
model = Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', LinearRegression(fit_intercept=False))])
model = model.fit(X_train, Y_train)
model.named_steps['linear'].coef_

pred3 = model.predict(np.asarray(df.horsepower).reshape(-1,1))
mse3 = mean_squared_error(np.asarray(df.mpg).reshape(-1,1),pred3)
print(mse3)

19.62663914673343


In [7]:
#Another way of doing it. 
np.random.seed(1)
train = np.random.choice(df.shape[0], 196, replace=False)
select = np.in1d(range(df.shape[0]), train)

import statsmodels.formula.api as smf
lm = smf.ols ('mpg~horsepower', data = df[select]).fit()
print( lm.summary())
preds = lm.predict(df)
square_error = (df['mpg'] - preds)**2
print( '--------Test Error for 1st order--------')
print( np.mean(square_error[~select]))

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Sun, 05 May 2019   Prob (F-statistic):           1.28e-42
Time:                        13:25:26   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

# Lab 5.3.2 Leave-One-Out Cross Validation

In [8]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

ols_fit = smf.ols ('mpg~horsepower', data = df).fit()
print(ols_fit.params)

glm_fit = sm.GLM.from_formula('mpg~horsepower', data = df).fit()
print(glm_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64
Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [9]:
from sklearn.model_selection import KFold, cross_val_score
x = pd.DataFrame(df.horsepower)
y = df.mpg

model = LinearRegression().fit(x,y)
print(model.intercept_)
print(model.coef_)

39.93586102117047
[-0.15784473]


In [10]:
# LOOCV use folds equal to # of observations, therefor k_fold is used with this number of folds. 
k_fold = KFold(n_splits=x.shape[0]) 
test= cross_val_score(model,x,y,cv=k_fold,scoring='neg_mean_squared_error', n_jobs = -1)
print(np.mean(-test))

24.23151351792923


In [11]:
# Repeats the LOOCV for increasing polynomials. 
A = []
for porder in range(1,6): 
    model = Pipeline([('poly', PolynomialFeatures(degree = porder)),('linear',LinearRegression())])
    k_fold = KFold(n_splits=x.shape[0]) 
    test= cross_val_score(model,x,y,cv=k_fold,scoring='neg_mean_squared_error', n_jobs = -1)
    A.append(np.mean(-test))
    
print(A)

[24.231513517929226, 19.248213124489393, 19.334984064131373, 19.424430309564396, 19.03321677574123]


# Opgave 5.3.3 K-folds 

In [12]:
np.random.seed(17)
A = []
for porder in range(1,11): 
    model = Pipeline([('poly', PolynomialFeatures(degree = porder)),('linear',LinearRegression())])
    k_fold = KFold(n_splits=10) 
    test= cross_val_score(model, x, y, cv=k_fold, scoring='neg_mean_squared_error', n_jobs = -1)
    A.append(np.mean(-test))
    
print(A)

[27.439933652339857, 21.23584005580212, 21.336606183328797, 21.353886994222318, 20.905630543164584, 20.79963125835415, 20.953366101158057, 21.0771808083011, 21.036717405054468, 20.980983108694186]


# Opgave 5.3.4 Bootstrap 

In [14]:
portfolio = pd.read_csv("C:\\Users\\diteh\\Documents\\Biomedicinsk Teknologi\\3. semester\\Decision Support System\\Lab\\Portfolio.csv", header = 0)
portfolio.head()

,Unnamed: 0,X,Y
0,1,-0.895251,-0.234924
1,2,-1.562454,-0.885176
2,3,-0.417090,0.271888
3,4,1.044356,-0.734198
4,5,-0.315568,0.841983


In [5]:
#Defining Alpha function - see book page 194
def alpha_fn(data,index): 
    X=data.X[index]
    Y=data.Y[index]
    return (np.var(Y)- np.cov(X,Y)[0,1]) / (np.var(X) + np.var(Y) -2* np.cov(X, Y)[0,1])


In [6]:
#Using alpha function on the data: 
alpha_fn(portfolio,list(range(0, 100)))

0.5766511516104118

In [6]:
# Inspiration found at: 
# https://github.com/qx0731/ISL_python/blob/master/Chapter_5_sec_3.1_3.4.ipynb

In [7]:
# randomly selecting 100 observations from the range 1 to 100, with replacement. 
# equivalent to constructing a new bootstrap data set and recomputing \alpha^hat based on the new data set.
np.random.seed(1)
alpha_fn(portfolio, np.random.choice(range(0,100), size =100, replace = True))

0.45048204924559027

In [8]:
# Defining bootstrap function - see earlier link. 
def boot_python(data, input_fun, iteration):
    n = portfolio.shape[0]
    idx = np.random.randint(0, n, (iteration, n))
    stat = np.zeros(iteration)

    for i in range(len(idx)):
        stat[i] = input_fun(data, idx[i])
    
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}

In [16]:
# Using the Bootstrap function on the data, based on the alpha function. Making 1000 estimates.  
boot_python(portfolio, alpha_fn, 1000)
# STD is calculated to 0.0928, and the mean (alpha) to 0.576

{'Mean': 0.5755541411244204, 'STD': 0.09340608028857275}